In [1]:
import json
import pymongo
import tweepy

with open('consumer_key.txt', 'r') as f:
    consumer_key =  f.read()
f.closed

with open('consumer_secret.txt', 'r') as f:
    consumer_secret = f.read()
f.closed

with open('access_key.txt', 'r') as f:
    access_key = f.read()
f.closed

with open('access_secret.txt', 'r') as f:
     access_secret = f.read()
f.closed

# Authentication
auth = tweepy.OAuthHandler(consumer_key[:-1], consumer_secret[:-1])
auth.set_access_token(access_key[:-1], access_secret[:-1])
api = tweepy.API(auth)

In [2]:
# Test access

USER_NAME = "nurietta_17"
user = api.get_user(id=USER_NAME)
user = user._json
print (user['verified'])
#print(json.dumps([user._json], sort_keys=True, indent=4, separators=(',',':')))

False


In [20]:
# Connect with mongodb

try:
    with open("credentials.txt", 'r', encoding='utf-8') as f:
        [name,password,url,dbname]=f.read().splitlines()
        conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
    print ("Connected successfully!!!")
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 
    

    
db = conn['tweempact']
collection = db['tweets']

Connected successfully!!!


In [11]:
# Find 100 actual tweets located in Spain to store its user in mlab

#collection = db['tweet_users']
    
search_results =api.search(q='*', geocode='39.3262345,-4.8380649,750km', count=100, lang='es')

json_users = []
print(len(search_results))
for i in search_results:
    document = i._json
    #collection.insert_one(document)
    document['id'] = str(document['id'])
    json_users.append(document)
    
print(len(json_users))

98
98


In [12]:

new_users = new_users + json_users
print(len(new_users))

NameError: name 'new_users' is not defined

In [27]:
# Get all the tweets stored in the collection "tweet_users" in mlab

cursor = collection.find()
print(cursor)

# Obtain json 
json_docs = []
for doc in cursor:
    doc['_id'] = str(doc['_id'])
    json_docs.append(doc)

In [28]:
# Data format (uploaded to mlab in "tweet_users" collection)

print(json.dumps([json_docs[0]], sort_keys=True, indent=4, separators=(',',':')))

[
    {
        "_id":"5bd4a79db4c0e32bc7f902b1",
        "contributors":null,
        "coordinates":null,
        "created_at":"Sat Oct 27 17:59:42 +0000 2018",
        "entities":{
            "hashtags":[],
            "symbols":[],
            "urls":[],
            "user_mentions":[
                {
                    "id":1031950877387620358,
                    "id_str":"1031950877387620358",
                    "indices":[
                        0,
                        11
                    ],
                    "name":"spooky yume\ud83d\udd77",
                    "screen_name":"__yumecchi"
                }
            ]
        },
        "favorite_count":0,
        "favorited":false,
        "geo":null,
        "id":1056244057838641152,
        "id_str":"1056244057838641152",
        "in_reply_to_screen_name":"__yumecchi",
        "in_reply_to_status_id":1056234978965626881,
        "in_reply_to_status_id_str":"1056234978965626881",
        "in_reply_to_user_id":103

In [45]:
# Get all the usernames

user_names = []

for doc in json_users:
    user_names.append(doc['user']['screen_name'])

In [46]:
print(len(user_names))

100


In [30]:
# Number of unique users found

import numpy as np

user_names = set(user_names)
print(len(user_names))

278


In [47]:
# Get 25 tweets of each user until 30th of September of 2018

import time

#MAX_ID = 1046439836939415562 # 30 of September
MAX_ID = 1055786926802898945 # 26 of October

all_tweets = []

for user in user_names:
    
    try:
        new_tweets = api.user_timeline(screen_name=user, count=25, max_id = MAX_ID)
        
    except tweepy.RateLimitError:
        print('Rate limit error with user: %s' % user)
        time.sleep(120)
        continue
    
    except tweepy.TweepError:
        print('Tweep error with user: %s' % user)
        time.sleep(60)
        continue
        
    print(user)
        
    all_tweets.extend(new_tweets)
    
    time.sleep(60)
   

antpogar
DarylMaldonado
menesess_pau
Ura2019
commiesans
Yerbaalawita
Kaytxar
I3Aunizar
GracielaLpez5
Nurya1952
Angelmg90
_asgxv
AlvatrosPubli
thefromthetree
irinaa_ce
AlfredoRstrepo
annasazzle
EfectoCognitivo
morbologo75
igsan_madr
gabinavarros
cmnoya60
gp_victor89
salvaglnet
Mildred12279353
ElChikitoGH
MonWoW_
FerreroEU
ciudadanodmund
angelapeke16
minnervaa
chag_hai
LeiretxoP
firecrackerx
AnaJorgegon
Jusesantos28
RotaDatos
victordejotamar
Postematico
JaraTirado
cristinetavg
Nutriguia
Victor_Corso
CarolinaSarrion
luciagarcia02
aragonradio
juanflm
AjXirivella
inmaCR
AngelLargo1970
Bea_sotelo
ShadowsNessa
Infinito81
DiscosRuidosos
pato_albert
mercedesslopez8
pipio44
vmezadelc
KateAusten5
Egarcamp111
InboxPublicitat
esguilu
SoniaGonzlezSa2
anndrxa_
angusticlavio
elvictorjuan
COMENTANDOGHVI2
Salem_Lombardi
liberal_mirada
anaisleali
LaNaveMadrid
negociosyempren
Themeele
jpa1968fr
jaimitollamas
EJASOabogados
AEHCOS
firelovesrain
JoseLui24493852
cafes_eguia
ElChikitoGH
SenorSherry
JAMonzon
Gi

In [48]:
# Number of tweets from users located in Spain until 30th of September of 2018

print(len(all_tweets))    

2353


In [6]:
# Get tweet id from a known date

n = api.user_timeline(screen_name='nachomartin83', count=30)
for tweet in n:
    print(tweet._json['id'])
    print(tweet._json['created_at'])

1061533081197195265
Sun Nov 11 08:16:23 +0000 2018
1061298282637520896
Sat Nov 10 16:43:23 +0000 2018
1061052676283105281
Sat Nov 10 00:27:26 +0000 2018
1060997519805681664
Fri Nov 09 20:48:15 +0000 2018
1060987545213898752
Fri Nov 09 20:08:37 +0000 2018
1060976558532427778
Fri Nov 09 19:24:58 +0000 2018
1060517801033641984
Thu Nov 08 13:02:01 +0000 2018
1060292664665432065
Wed Nov 07 22:07:25 +0000 2018
1060292451523534848
Wed Nov 07 22:06:34 +0000 2018
1060267026424496128
Wed Nov 07 20:25:32 +0000 2018
1060262520341045248
Wed Nov 07 20:07:38 +0000 2018
1059964276847337472
Wed Nov 07 00:22:31 +0000 2018
1059947383663067137
Tue Nov 06 23:15:23 +0000 2018
1059910388463079424
Tue Nov 06 20:48:23 +0000 2018
1058155246130749442
Fri Nov 02 00:34:04 +0000 2018
1057010726768910338
Mon Oct 29 20:46:10 +0000 2018
1056665912684806144
Sun Oct 28 21:56:00 +0000 2018
1056665156338495488
Sun Oct 28 21:52:59 +0000 2018
1056510174675025920
Sun Oct 28 11:37:09 +0000 2018
1056176058700042240
Sat Oct 27 

In [50]:
# Store all tweets found in mlab "tweets" collection

collection = db['tweets']

for tweet in all_tweets:
    collection.insert_one(tweet._json)

In [4]:
# ************************************************************************************************************

# Scrapping actual tweets to get some usernames:

some_tweets = []
import time

In [5]:
for i in range(100):
    search_results =api.search(q='*', geocode='39.3262345,-4.8380649,750km', count=100, lang='es')

    for i in search_results:
        document = i._json
        document['id'] = str(document['id'])
        some_tweets.append(document)
    
    time.sleep(10)

In [6]:
print(len(some_tweets))

9879


In [10]:
user_names = []
verified = []

In [11]:
for doc in some_tweets:
    user_names.append(doc['user']['screen_name'])
    if doc['user']['followers_count'] > 5000 and doc['user']['followers_count'] < 100000:
        verified.append(doc['user']['screen_name'])

In [12]:
print(len(user_names))
print(len(verified))

9879
1056


In [70]:
set_user_names = set(user_names)
print(len(set_user_names))

7841


In [13]:
set_verified = set(verified)
print(len(set_verified))

675


In [14]:
def get_10tweets(username):
    
    tweets_10 = []
    count = 11
    MAX_ID = 1061533081197195265 #11 November
    aux = 0
    
    while len(tweets_10) < 11:
        print('.', end='')
        try:
            tweets = api.user_timeline(screen_name=username, count=count, max_id = MAX_ID)
        except tweepy.TweepError:
            return 0, tweets_10
        
        for tweet in tweets:
            doc = tweet._json
            doc['id'] = str(doc['id'])
            
            if doc not in tweets_10 and not doc['text'].startswith('RT '):
                tweets_10.append(doc)
            else:
                MAX_ID = doc['id']
            
            if len(tweets_10) == 11:
                break
        if aux > 10:
            return 2, tweets_10
        aux += 1
    
    return 1, tweets_10

In [48]:
#user_names = user_names[0:5]

In [15]:
set_user_names = set_verified
print(len(set_user_names))

675


In [16]:
collection = db['users_noRT']
cursor = collection.find()

set_new_users = []
repeated = []

for doc in cursor:
    if doc['name'] in set_user_names:
        repeated.append(doc['name'])

for u in set_verified:
    if u not in repeated:
        set_new_users.append(u)

In [17]:
print(len(set_new_users))
print(set_new_users)

633
['aplausos', 'photoespana', 'Toliol', 'jgarciabuitron', 'CE4reputation', 'madresfera', 'pacomeroo', 'CarrascoAntonia', 'alfredo15489', 'robertobgdoc', 'Ainhoa_superman', '24clm', 'PedroBDonoso', 'AytoAlcalaH', 'MariaTabarnia', 'CoroBrothers', 'finect', 'IPesqueras', 'Lady_Madrid1978', 'BenetBeca', 'AwesomeeIris', 'christinalosada', 'dondeestawally7', 'TNOradio', 'marcgarrigasait', 'NotaVeraz', 'MissMacGuffin', 'fray_fanatic', 'jxvig', 'VMatterfilm', 'huelvaya', 'gilstefania', 'BuddhaSeeds', 'Albiazules', 'pprimor', 'novelasHispania', 'geologoenapuros', 'inesluis', 'LuisDeveraux', 'electo_mania', 'Autores_noveles', 'CENIEH', 'ciospain', 'veredes', 'GrupoEnergia16', 'ITReseller_ITDM', 'JordiEstradaB', 'devilishgames', 'gasteizhoy', 'aragonradio', 'RecruitErasmus', 'batiralibex35', 'Daurmith', 'CorunaNoticias', 'enfermeratweet', 'Laura0Garcia', 'Sergio_Cala', 'METALOCUS', 'UNICEFAndalucia', 'brunoalemany', 'matiasjurisich', 'RollingStonx', 'ByGamerEspHD', 'silviaalava', 'NuriaCSopena'

In [18]:
list_of_tweets = []
status_0_users = []
status_2_users = []

In [19]:
import time
for name in set_new_users:
    print(name)
    status, tweets = get_10tweets(name)
    time.sleep(30)
    if status == 1:
        list_of_tweets.extend(tweets)
    elif status == 0:
        print("Status : 0")
        status_0_users.append(name)
    else:
        print("Status : 2")
        status_2_users.append(name)
        list_of_tweets.extend(tweets)

aplausos
..photoespana
..Toliol
..jgarciabuitron
..CE4reputation
..madresfera
...pacomeroo
......CarrascoAntonia
...alfredo15489
..robertobgdoc
...Ainhoa_superman
..24clm
..PedroBDonoso
.AytoAlcalaH
..MariaTabarnia
..CoroBrothers
..finect
.IPesqueras
.Lady_Madrid1978
.BenetBeca
............Status : 2
AwesomeeIris
............Status : 2
christinalosada
.......dondeestawally7
.TNOradio
.marcgarrigasait
..NotaVeraz
............Status : 2
MissMacGuffin
.fray_fanatic
...jxvig
.VMatterfilm
......huelvaya
.gilstefania
.BuddhaSeeds
..Albiazules
..pprimor
....novelasHispania
..geologoenapuros
.inesluis
............Status : 2
LuisDeveraux
.electo_mania
..Autores_noveles
..CENIEH
......ciospain
.......veredes
......GrupoEnergia16
.ITReseller_ITDM
..JordiEstradaB
..devilishgames
....gasteizhoy
.aragonradio
.RecruitErasmus
.batiralibex35
...Daurmith
..CorunaNoticias
.enfermeratweet
.Laura0Garcia
..Sergio_Cala
...........METALOCUS
..UNICEFAndalucia
..brunoalemany
..matiasjurisich
.RollingStonx
.....

..mamparasblau
.itziarwxst
..KikeMlaga
........SigueLumen
....onthespot
..cruzdeguiaser
.RMadridistaReal
...CajaGranadaOBS
..TosfritOficial
.cocinadeenloqui
.epmediterraneo
...DuroBelinda
............Status : 2
SaraHispania
............Status : 2
Nortexpres
.Innobasque
...JoanSeguidor
..VBegoa
..podemosmurcia
....AVEBIOM
...dinahosting
..CartagenaFS
..lapaseata
............Status : 2
learningenlish
............Status : 2
frmat
........olliver2013
............Status : 2
poilemutawi
...sorianoticias
.baratukin
.c_Emprende_Alc
..DefensorAndaluz
.cristinabosca
...uvemayuscula
..mundoficcion
....ProfdonPardino
..locosxlosviajes
..RS_Gimnastica
............Status : 2
cla1112
............Status : 2
gamificatuaula
....SanlucarInfo
.EduardoBandera1
..exp_ecodigital
.BCD_barcelona
.jatirado
............Status : 2
Fromerofoto
..CiudadDBadajoz
..jaime_delbarrio
...........Esp_Marvelita
.enisa
.santiagonavajas
.patriziamiani
............Status : 2
HollyMolly
.Legalitas_ES
.HoffmannMethod
....JordiV

In [21]:
print(len(list_of_tweets))

6487


In [22]:
collection = db['users_noRT']
for name in set_new_users:
    doc = {}
    doc['name'] = name
    collection.insert_one(doc)

In [23]:
collection = db['tweets_noRT']
for tweet in list_of_tweets:
    collection.insert_one(tweet)

In [124]:
set_usernames_1 = set_user_names

In [125]:
print(len(set_usernames_1))

486


In [165]:
set_tweets_1 = list_of_tweets

In [166]:
print(len(set_tweets_1))

3927


In [181]:
set_usernames_2 = []

In [182]:
for name in set_user_names:
    if name not in set_usernames_1:
        set_usernames_2.append(name)

In [183]:
print(len(set_usernames_2))

1035


In [185]:
collection = db['users_noRT']
for name in set_usernames_2:
    doc = {}
    doc['name'] = name
    collection.insert_one(doc)

In [186]:
set_tweets_2 = []

In [187]:
for tweet in list_of_tweets:
    if tweet not in set_tweets_1:
        set_tweets_2.append(tweet)

In [188]:
print(len(set_tweets_2))

6679


In [189]:
collection = db['tweets_noRT']
for tweet in set_tweets_2:
    collection.insert_one(tweet)

In [190]:
collection = db['users_noRT']
cursor = collection.find()

for doc in cursor:
    if doc['name'] in status_0_users:
        collection.delete_one({'name': doc['name']})

In [191]:
print(len(status_0_users))

46


In [192]:
print(len(status_2_users))

537


In [8]:
#************************************************************************************
# Scrap some verified users
set_new_users = []

In [9]:
with open('verified-users.txt') as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        set_new_users.append(line)
        

In [10]:
print(len(set_new_users))

199
